In [ ]:
import numpy as np
from collections import defaultdict

#Creating an Artificial Neural Network class


class NeuralNetwork:
    def __init__(self, x, y, value):
        self.x = x
        self.y = y
        self.value = value
        
    def __repr__(self):
        return "x:{0} y:{1} v:{2}".format(self.x, self.y, self.value)

In [ ]:
#define the gameboard
class gameboard:
    x = 1  #differentiate the two player
    o = -1
    def __init__(self, state, next_move=1):
        self.board = state
        self.board_size = state.shape[0] #0 means place still available
        self.next_move = next_move
        
    def result(self):
        row_sum = np.sum(self.board, 0)
        col_sum = np.sum(self.board, 1)
        diag_sum1 = self.board.trace()
        diag_sum2 = self.board[::-1].trace()
        
        s1 = any(row_sum == self.board_size) #different situation
        s2 = any(col_sum == self.board_size)
        s3 = (diag_sum1 == self.board_size)
        s4 = (diag_sum2 == self.board_size)
        

        if s1 or s2 or s3 or s4:
            return self.x
        
        s1 = any(row_sum == -self.board_size)
        s2 = any(col_sum == -self.board_size)
        s3 = (diag_sum1 == -self.board_size)
        s4 = (diag_sum2 == -self.board_size)
        

        if s1 or s2 or s3 or s4:
            return self.o
        if np.all(self.board != 0):
            return 0
        return None
       
    
    def game_over(self):
        return self.result() is not None
    
    
    def game_continue(self, move):
        # If it is not the current players turn, can not move
        if move.value != self.next_move:
            return False
        
        # If move is outside of board, can not move
        if ((not (0 <= move.x < self.board_size)) or
            (not (0 <= move.y < self.board_size))):
            return False
        
        # If (x, y) is not occupied, ok, otherwise not.
        return self.board[move.x, move.y] == 0
        
        
    def move(self, move):
        if not self.game_continue(move):
            raise ValueError(format(move, self.board))

        new_board = np.copy(self.board)
        new_board[move.x, move.y] = move.value
            
        return gameboard(new_board, self.next_move*-1)
        
    def legal_actions(self):
        indices = np.where(self.board == 0)
        return [
            NeuralNetwork(c[0], c[1], self.next_move)
            for c in list(zip(indices[0], indices[1]))
        ]

In [ ]:
#define the MCTS node


class MCTSNode:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.results = defaultdict(int)
        self.n_visits = 0
        self._untried_actions = None
        
    def untried_actions(self):
        if self._untried_actions is None:
            self._untried_actions = self.state.legal_actions()
        return self.state.legal_actions()
    
    def Q(self):
        n_wins = self.results[self.parent.state.next_move]
        n_losses = self.results[-1 * self.parent.state.next_move]
        return n_wins - n_losses
     
    def N(self):
        return self.n_visits
    
    def expand(self):
        action = self.untried_actions().pop()
        next_state = self.state.move(action)
        child = MCTSNode(next_state, parent=self)
        self.children.append(child)
        return child
    
    def terminal_node(self):
        return self.state.game_over()
    
    def roll_out(self):
        current_rollout_state = self.state
        while not current_rollout_state.game_over():
            possible_moves = current_rollout_state.legal_actions()
            action = self.rollout_policy(possible_moves)
            current_rollout_state = current_rollout_state.move(action)
        return current_rollout_state.result()
            
    
    def backpropagate(self, result):
        self.n_visits += 1
        self.results[result] += 1
        if self.parent:
            self.parent.backpropagate(result)
    
    def is_fully_expanded(self):
        return len(self.untried_actions()) == 0
    
    def best_child(self, c_param=1.4):
        choices_weights = [
            (c.Q() / c.N()) + c_param * np.sqrt((2*np.log(self.N()) / c.N()))
            for c in self.children
        ]
        return self.children[np.argmax(choices_weights)]
        
    def rollout_policy(self, possible_moves):
        return possible_moves[np.random.randint(len(possible_moves))]

In [ ]:
#implement the MCTS
class MCTS:
    def __init__(self, node):
        self.root = node
        
    def best_action(self, n_simulations): 
        for _ in range(0, n_simulations):
            v = self.tree_policy()
            reward = v.roll_out()
            v.backpropagate(reward)
        
        return self.root.best_child(c_param=0.)
    
    def tree_policy(self):
        current_node = self.root
        while not current_node.terminal_node():
            if not current_node.is_fully_expanded():
                return current_node.expand()
            else:
                current_node = current_node.best_child()
        return current_node

In [ ]:
#play the game

#current_state = np.zeros((3,3))  #if we want to play with 3 by 3


#if we want to play with 4 by 4
current_state = np.zeros((4,4))

next_move = 1 #set AI as 1
print("AI: -1, Human:1, Available place;0")
current_board = gameboard(state = current_state, next_move=next_move)
while not current_board.game_over():
    if next_move == -1: #human move
        while True:
            user_input = input()
            coords = user_input.strip().split(sep=",")
            x = int(coords[0])
            y = int(coords[1])
            if current_board.game_continue(NeuralNetwork(x, y, next_move)):
                current_state[x, y] = -1
                current_board = gameboard(state = np.copy(current_state), next_move=-1*next_move)
                break
            else:
                print("The place is NOT available!!!") 

        print("User move:")
        print(current_state)
            
    else:
        root = MCTSNode(state = current_board)
        mcts = MCTS(root)
        best_node = mcts.best_action(1000)
        current_state = best_node.state.board
        current_board = gameboard(state = np.copy(current_state), next_move=-1*next_move)
   
        print("AI move:")
        print(current_state)
    next_move *= -1
print("Game Over")


AI: -1, Human:1, Available place;0
AI move:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]
2,1
User move:
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  0.  1.]]
AI move:
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  1.  1.]]
2,3
User move:
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0. -1.  0. -1.]
 [ 0.  0.  1.  1.]]
AI move:
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0. -1.  0. -1.]
 [ 0.  1.  1.  1.]]
